# Improved Benchmark Processing with NF1-dataset using CytoSnake's `cp_process_singlecells` Workflow

This notebook uses the benchmark data obtained through the application of CytoSnake's `cp_process_singlecells` workflow. The collected data is then compiled into a unified benchmark profile, allowing comparisons with other generated benchmark profiles in this repo.

This benchmark profile was generated  by using the [`NF1-data`](https://github.com/WayScience/nf1_cellpainting_data)

**NOTE**: This benchmark profile only profiles the single-cell pipeline found [here](https://github.com/WayScience/nf1_cellpainting_data/blob/main/3.processing_features/2.pycytominer_singlecell_pipelines.ipynb).

The bulk benchmarking profile will be found in the `all-benchmarks/nf1_bulk_cp-process-singlecells_benchmarks` directory

In [1]:
import sys
import json
import pathlib
import subprocess
from datetime import datetime

import pandas as pd

sys.path.append("../../../")
from src.benchmark_utils import get_benchmark_files

## Generating CytoSnake `cp_process_singlecells` Benchmark Profile with NF1 Data

In this section, we are generating a benchmark profile for CytoSnake's `cp_process_singlecells` applied to the NF1 dataset. For additional information on the NF1 dataset, detailed documentation is available in its repository and can also be found in the controls directory of this repository.

In [2]:
# parameters
CWD_PATH = pathlib.Path(".").resolve()
BENCHMARK_DIR = pathlib.Path("./benchmarks/").resolve(strict=True)

In [3]:
# loading in file size data:
with open("./file_size.json", mode="r") as stream:
    file_sizes = json.load(stream)

# this is in MB
print(file_sizes)

{'Plate_1': 4.727, 'Plate_3': 551.629, 'Plate_3_prime': 444.938, 'Plate_4': 222.945, 'Plate_2': 30.598}


In [4]:
# converting .bin files into .json files (~2min)
# skip this block if the json files are already in the benchmarks/ directory
for bin_path in get_benchmark_files(BENCHMARK_DIR, ext="bin"):
    json_out = BENCHMARK_DIR / f"{bin_path.stem}.json"

    # # executing memray to convert bin files into json files
    memray_stats = subprocess.run(
        [
            "memray",
            "stats",
            "--json",
            "--output",
            str(json_out),
            "--force",
            str(bin_path),
        ],
        capture_output=True,
        check=True,
    )

    # stdout message
    print(
        f"{bin_path.relative_to(CWD_PATH)} was successfully converted into {json_out.relative_to(CWD_PATH)}"
    )

benchmarks/Plate_3_nf1_feature_select_benchmarks.bin was successfully converted into benchmarks/Plate_3_nf1_feature_select_benchmarks.json
benchmarks/Plate_3_prime_nf1_analysis_normalize_benchmarks.bin was successfully converted into benchmarks/Plate_3_prime_nf1_analysis_normalize_benchmarks.json
benchmarks/Plate_4_nf1__feature_select_benchmarks.bin was successfully converted into benchmarks/Plate_4_nf1__feature_select_benchmarks.json
benchmarks/Plate_2_nf1_feature_select_benchmarks.bin was successfully converted into benchmarks/Plate_2_nf1_feature_select_benchmarks.json
benchmarks/Plate_1_nf1_analysis_normalize_benchmarks.bin was successfully converted into benchmarks/Plate_1_nf1_analysis_normalize_benchmarks.json
benchmarks/Plate_1_nf1_feature_select_benchmarks.bin was successfully converted into benchmarks/Plate_1_nf1_feature_select_benchmarks.json
benchmarks/feature_select_benchmarks.bin was successfully converted into benchmarks/feature_select_benchmarks.json
benchmarks/Plate_2_nf

In [5]:
# using all json files to compile benchmark profile
raw_benchmark_data = []
for json_path in get_benchmark_files(BENCHMARK_DIR, ext="json"):
    # collecting data from just file name
    plate_name = json_path.stem.split("_nf1_")[0]
    try:
        file_size = file_sizes[plate_name]
    except KeyError:
        file_size = "0"

    # opening json file to extract benchmark information
    with open(json_path, encoding="utf-8", mode="r") as contents:
        benchmark_data = json.load(contents)

        # accessing to all metadata from benchmarks
        meta_data = benchmark_data["metadata"]
        selected_data = {
            "pid": meta_data["pid"],
            "process_name": process_name,
            "input_data_name": plate_name,
            "start_time": datetime.strptime(meta_data["start_time"], tformat),
            "end_time": datetime.strptime(meta_data["end_time"], tformat),
            "time_duration": (
                datetime.strptime(meta_data["end_time"], tformat)
                - datetime.strptime(meta_data["start_time"], tformat)
            ).total_seconds(),
            "total_allocations": int(meta_data["total_allocations"]),
            "peak_memory": round(int(meta_data["peak_memory"]) / 1024**2, 3),
            "file_size": file_sizes[plate_name],
        }
        # append to list
        raw_benchmark_data.append(selected_data)

NameError: name 'process_name' is not defined

In [ ]:
# generating benchmark profile and saving it
benchmark_profile = pd.DataFrame(data=raw_benchmark_data)
benchmark_profile

""
